# Used Car Price Estimator

### Summary
[Auto.dev](https://www.auto.dev/) is an API with recent car listing data. Data on Toyota Camrys (n~1000) was gathered and a model was developed to predict car price. Multiple linear regression was used with year, mileage, and trim as independent predictors. The model's fitted values were used to identify cars with the largest discounts (residuals).

### Next Steps
* A daily check-in that can generate discount prices on new cars
* A tracker of new listing data for seasonal analysis

### Code:
#### Part 1: Setup

In [19]:
# Jupyter occasionally has errors with retrieving packages installed in other places/environments
!{sys.executable} -m pip install --upgrade geopy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 1.1 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 840.4 kB/s eta 0:00:00 0:00:01


In [21]:
# Import relevant libraries
import requests
import json
import pandas as pd
import numpy as np
from pandas import json_normalize
import statsmodels.api as sm
import matplotlib.pyplot as plt
import sys
import math
from scipy.stats import chi2
import seaborn as sns
from geopy import distance

#### Part 2: Call API for Data
Data on ~1000 Toyota Camry listings was gathered. Cars before 2016 were not considered. All mileages were considered for the model building. There were four trims used for model building (SE, LE, XSE, XLE). All other trims had low and unreliable counts and were thus excluded.

In [5]:
# Base URL for API requests
url = "https://auto.dev/api/listings"
# General parameters for model trainings
params = {
    'apikey': 'ZrQEPSkKbWF4bWVsbmlrYXNAZ21haWwuY29t',
    'sort_filter': 'created_at:desc',
    'year_min': 2016,
    'make': 'Toyota',
    'model': 'Camry',
    #'city': 'Boston',
    #'state': 'MA',
    #'location': 'Boston, MA',
    'longitude': -71.058884,
    'latitude': 42.360081,
    'radius': 150,
    'transmission[]': 'automatic',
    'exclude_no_price': 'true'
    # Remove or properly set empty parameters
}

# Each page has 20 entries therefore 50 pages need to be collected
pages = list(range(1, 51))

# Initialize an empty list to store all records
all_data = []

# Loop over each page, fetch data, and append to the list
for page in pages:
    params['page'] = page
    response = requests.get(url, params=params)
    result = response.json()
    
    if 'records' in result:
        all_data.extend(result['records'])
    else:
        print(f"Warning: 'records' key not found on page {page}")

# Normalize the JSON data into a DataFrame
df = json_normalize(all_data)

# Now `df` contains all the records from the fetched pages
df  # To display the first few rows of the DataFrame

,id,vin,displayColor,year,make,model,price,mileage,city,lat,...,trackingParams.remoteDealerId,trackingParams.dealerName,trackingParams.remoteSku,trackingParams.experience,trackingParams.rooftopUniqueName,trackingParams.rooftopUuid,trackingParams.dealerUniqueName,trackingParams.dealerUuid,trackingParams.dealerGroupUniqueName,trackingParams.dealerGroupUuid
0,295506920,4T1G11AK2NU693182,Black,2022,Toyota,Camry,"$24,998","51,123 Miles",Marlborough,42.3336,...,448313,None,1244,local,None,None,None,None,None,None
1,295506670,4T1DBADK7SU007709,Supersonic Red,2025,Toyota,Camry,"$41,523",New,Brockton,42.0550,...,54419,None,,local,None,None,None,None,None,None
2,295505121,4T1BF1FK8HU633822,Silver,2017,Toyota,Camry,"$19,875","63,131 Miles",Rochester,43.2538,...,305772,None,633822,local,None,None,None,None,None,None
3,295504992,4T1BF1FK4GU239121,Black,2016,Toyota,Camry,"$12,495","120,739 Miles",Putnam,41.9247,...,370282,None,3206,local,None,None,None,None,None,None
4,295482290,4T1DBADK7SU508972,Midnight Black Metallic,2025,Toyota,Camry,"$33,894",New,Manchester,42.9518,...,175104,None,SU508972,local,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788,274973890,4T1BF1FK7HU341689,Yellow,2017,Toyota,Camry,"$13,495","86,067 Miles",Patchogue,40.7827,...,435091,None,37009,local,None,None,None,None,None,None
789,274186249,4T1C11AK1MU562127,White,2021,Toyota,Camry,"$17,995","97,853 Miles",New Haven,41.2872,...,292347,None,562127,local,None,None,None,None,None,None
790,273201923,4T1B11HK3JU514108,Burgundy,2018,Toyota,Camry,"$17,500","55,000 Miles",Seabrook,42.8749,...,153486,None,,local,None,None,None,None,None,None
791,273198730,4T1C11BK0NU052867,White,2022,Toyota,Camry,"$24,999","46,482 Miles",North Attleboro,41.9580,...,280597,None,,local,None,None,None,None,None,None


In [27]:
pd.set_option('display.max_rows', 100)
df.iloc[1]

id                                                                              295506670
vin                                                                     4T1DBADK7SU007709
displayColor                                                               Supersonic Red
year                                                                                 2025
make                                                                               Toyota
model                                                                               Camry
price                                                                             $41,523
mileage                                                                               New
city                                                                             Brockton
lat                                                                                42.055
lon                                                                               -71.056
primaryPho

#### Part 3: Clean data
Three car features are included in this model:
- Model Year
- Mileage
- Body Trim

In [9]:
def convert_mileage(value):
    value = value.strip()
    if value.lower() == 'new':
        return 0
    try:
        # Remove non-numeric characters and convert to int
        return int(value.replace(',', '').replace(' Miles', ''))
    except ValueError:
        # Handle unexpected cases
        return None

def convert_year(value):
    value = 2025 - value + 1
    return math.log(value)

def convert_price(value):
    if isinstance(value, str):
        value = value.replace('$', '').replace(',', '')
        try:
            return float(value)
        except ValueError:
            return None
    return None

In [115]:
X = df[['year', 'mileageUnformatted', 'priceUnformatted', 'trim']]

X = X.dropna()

In [121]:
X_mod.iloc[1]

const                     1.000000
mileageUnformatted    63131.000000
trim_SE                   0.000000
trim_XLE                  1.000000
trim_XSE                  0.000000
year_2016                 0.000000
year_2017                 1.000000
year_2018                 0.000000
year_2019                 0.000000
year_2020                 0.000000
year_2021                 0.000000
year_2022                 0.000000
year_2023                 0.000000
year_2024                 0.000000
year                      2.197225
Name: 2, dtype: float64

In [119]:
# Define the trims you want to include
trims_to_filter = ['SE', 'LE', 'XSE', 'XLE']

# Filter out any unwanted trims
X_mod = X[X['trim'].isin(trims_to_filter)]
y = X_mod.pop('priceUnformatted')

# Convert remaining trims into categorical data types
X_mod['trim'] = X_mod['trim'].astype('category')
X_mod = pd.get_dummies(X_mod, columns=['trim'], drop_first=True)


X_mod = sm.add_constant(X_mod)

# Allow each model year to have an additional affect beyond the existing logarithmic effect
temp = X_mod['year']
X_mod = pd.get_dummies(X_mod, columns=['year'], drop_first=False)
X_mod.pop('year_2025')

X_mod.iloc[:, range(2,14)] = X_mod.iloc[:, range(2,14)].astype(int)
X_mod['year'] = temp.apply(convert_year)

/var/folders/zf/q6vckqdx64v8bb5_y2dtjvjw0000gn/T/ipykernel_73035/3822541066.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_mod['trim'] = X_mod['trim'].astype('category')
/var/folders/zf/q6vckqdx64v8bb5_y2dtjvjw0000gn/T/ipykernel_73035/3822541066.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0      1
2      0
3      0
4      1
6      0
      ..
787    1
788    0
789    0
791    0
792    0
Name: trim_SE, Length: 600, dtype: int64' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  X_mod.iloc[:, range(2,14)] = X_mod.iloc[:, range(2,14)].astype(int)
/var/folders/zf/q6vckqdx64v8bb5_y2dtjvjw0000gn/T/ipykernel_73035/3822541066.py:20

#### Part 4: Build model


In [123]:
regression_model = sm.GLM(y, X_mod, family=sm.families.Gaussian()).fit()

# Print the summary of the model
print(regression_model.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:       priceUnformatted   No. Observations:                  600
Model:                            GLM   Df Residuals:                      586
Model Family:                Gaussian   Df Model:                           13
Link Function:               Identity   Scale:                      3.8060e+06
Method:                          IRLS   Log-Likelihood:                -5389.9
Date:                Tue, 27 Aug 2024   Deviance:                   2.2303e+09
Time:                        11:11:18   Pearson chi2:                 2.23e+09
No. Iterations:                     3   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const               3.485e+04    180

#### Part 5: Evaluate model

This section calculates the estimated price of a car based on input features. Cars with large discounts (difference between predicted price and actual price) are identified.

In [125]:
new_car = pd.DataFrame({'const': [1], 
                        'mileageUnformatted': [39000],
                        'trim_SE': [0],
                        'trim_XLE': [1],
                        'trim_XSE': [0],
                        'year_2016': [0],
                        'year_2017': [0],
                        'year_2018': [1],
                        'year_2019': [0],
                        'year_2020': [0],
                        'year_2021': [0],
                        'year_2022': [0],
                        'year_2023': [0],
                        'year_2024': [0],
                        'year': math.log(8)
                       })
# Predict the price
regression_model.predict(new_car)

0    24195.205886
dtype: float64

In [129]:
regression_model.resid_pearson[regression_model.resid_pearson < -4000]

40    -4980.049150
99    -4600.982509
169   -4536.049150
184   -4766.786471
188   -5808.715945
373   -4740.249072
455   -4523.174079
dtype: float64

In [141]:
pd.set_option('display.max_rows', 100)
df.iloc[184]

id                                                                              295040237
vin                                                                     4T1BF1FK9GU540290
displayColor                                                    Celestial Silver Metallic
year                                                                                 2016
make                                                                               Toyota
model                                                                               Camry
price                                                                             $16,995
mileage                                                                      57,735 Miles
city                                                                             Johnston
lat                                                                               41.8647
lon                                                                              -71.4976
primaryPho

#### Part 6: Automate a daily newsletter

In [1]:
%run daily_newsletter.py

Page 1: All records are within the last 24 hours.
Found an entry older than 24 hours, stopping the loop.


/Users/maxmelnikas/Documents/Personal Projects/Cars/daily_newsletter.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_mod['estimate_price'] = X_mod.apply(calculate_price, axis=1)
/Users/maxmelnikas/Documents/Personal Projects/Cars/daily_newsletter.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_mod['discount'] = X_mod['estimate_price'] - X_mod['priceUnformatted']
/Users/maxmelnikas/Documents/Personal Projects/Cars/helper_functions.py:124: FutureWarning: The behavior of DataFrame concatenation w

## Appendix

In [1]:
from helper_functions import update_latest_logs

update_latest_logs()

In [5]:
from datetime import datetime, timezone
datetime.now(timezone.utc)

datetime.datetime(2024, 9, 6, 1, 16, 17, 736288, tzinfo=datetime.timezone.utc)